# Исследование поведения пользователей мобильного приложения для покупки продуктов питания

Цель: разобраться как  ведут себя пользователи приложения и как изменится их поведение после изменения шрифта в приложении.  
Задача: изучить воронки продаж и результаты ААВ эксперимента

План работы:  
1. загрузка данных
2. предобработка
3. анализ воронок продаж: 
    - построение воронки продаж 
    - исследование поведения пользователей (шаги совершаемые в приложении)
4. анализ результатов ААВ эксперимента
    - между контрольными группами
    - между экспериментальной и каждой контрольными группами
    - между экспериментальной и обьединенными контрольными группами


## Описание данных   

Каждая запись в логе — это действие пользователя, или событие.     
`EventName` — название события;      
`DeviceIDHash` — уникальный идентификатор пользователя;     
`EventTimestamp` — время события;     
`ExpId` — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.   


# Подготовка данных

In [1]:
import pandas as pd
import datetime
import numpy as np
import scipy.stats as stat
import math as m

import warnings
warnings.filterwarnings('ignore')
import plotly.express as px

# для просмотретра проекта на nbviewer.org ---------
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.io as pio
# ----------------------------------------------------

pd.options.display.max_colwidth = 190

грузим данные

In [2]:
data = pd.read_csv(
        'https://code.s3.yandex.net/datasets/logs_exp.csv', sep='\t')

функция предварительного форматированиия названий столбцов , убирающая пробелы по краям названия, и заменяющая пробелы внутри на "_"

In [3]:

def get_format_columns(dataframe):
    '''функция принимает датафрейм и возврашает форматированные названия столбцов: 
    (Основное)
    1. без пробелов слева и справа
    2. пробел внутри названия столбца заменяется на символ "_" 

    '''

    return list(
        map(
            lambda x: x
            .strip()
            .replace(' ', '_'),
            dataframe.columns
        )
    )

форматируем названия столбцов

In [4]:
data.columns = get_format_columns(data)
data.columns

Index(['EventName', 'DeviceIDHash', 'EventTimestamp', 'ExpId'], dtype='object')

Переименовыываем для удобства

In [5]:
data.rename(columns={'EventName': 'event_name',
                     'DeviceIDHash': 'user_id',
                     'EventTimestamp': 'event_timestamp',
                     'ExpId': 'group'},
            inplace=True)
data.columns

Index(['event_name', 'user_id', 'event_timestamp', 'group'], dtype='object')

Выводим информацию о таблице

In [6]:
for df, name in zip([data], ['data']):
    print(f'\ndataframe: \t{name}\n{"-"*25}'.upper())
    print(df.info())
    display(df.head(10))

    print(f'\nдубликатов: {df.duplicated().sum()}\n{". "*10}'.upper())

    print(f'\nпропуски\n{". "*10}'.upper())
    print(df.isna().sum())

    print('----'*20)


DATAFRAME: 	DATA
-------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244126 entries, 0 to 244125
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   event_name       244126 non-null  object
 1   user_id          244126 non-null  int64 
 2   event_timestamp  244126 non-null  int64 
 3   group            244126 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 7.5+ MB
None


,event_name,user_id,event_timestamp,group
0,MainScreenAppear,4575588528974610257,1564029816,246
1,MainScreenAppear,7416695313311560658,1564053102,246
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248
5,CartScreenAppear,6217807653094995999,1564055323,248
6,OffersScreenAppear,8351860793733343758,1564066242,246
7,MainScreenAppear,5682100281902512875,1564085677,246
8,MainScreenAppear,1850981295691852772,1564086702,247
9,MainScreenAppear,5407636962369102641,1564112112,246



ДУБЛИКАТОВ: 413
. . . . . . . . . . 

ПРОПУСКИ
. . . . . . . . . . 
event_name         0
user_id            0
event_timestamp    0
group              0
dtype: int64
--------------------------------------------------------------------------------


Устраняем дубликаты

In [7]:
data.drop_duplicates(inplace=True)

создаем столбец с датой и временем

In [8]:
data['datetime'] = (data['event_timestamp']
                    .apply(lambda x:
                           datetime
                           .datetime
                           .fromtimestamp(x)
                           )
                    )



data.head()

,event_name,user_id,event_timestamp,group,datetime
0,MainScreenAppear,4575588528974610257,1564029816,246,2019-07-25 13:43:36
1,MainScreenAppear,7416695313311560658,1564053102,246,2019-07-25 20:11:42
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248,2019-07-25 20:28:47
3,CartScreenAppear,3518123091307005509,1564054127,248,2019-07-25 20:28:47
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248,2019-07-25 20:48:42


и отдельно столбец с датой

In [9]:

data['date'] = pd.to_datetime(data['datetime'].dt.date)
data.dtypes

event_name                 object
user_id                     int64
event_timestamp             int64
group                       int64
datetime           datetime64[ns]
date               datetime64[ns]
dtype: object

In [10]:
data.head()

,event_name,user_id,event_timestamp,group,datetime,date
0,MainScreenAppear,4575588528974610257,1564029816,246,2019-07-25 13:43:36,2019-07-25
1,MainScreenAppear,7416695313311560658,1564053102,246,2019-07-25 20:11:42,2019-07-25
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248,2019-07-25 20:28:47,2019-07-25
3,CartScreenAppear,3518123091307005509,1564054127,248,2019-07-25 20:28:47,2019-07-25
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248,2019-07-25 20:48:42,2019-07-25


переименуем для удобства группы 246, 247, 248 как А, А_1, B соответсвенно

In [11]:
def rename_group(arg):
    '''заменяет номера групп на буквы'''
    if arg == 246:
        return 'A'
    elif arg == 247:
        return 'A_1'
    else:
        return 'B'

In [12]:
data['group'] = data['group'].apply(rename_group)
data.head()

,event_name,user_id,event_timestamp,group,datetime,date
0,MainScreenAppear,4575588528974610257,1564029816,A,2019-07-25 13:43:36,2019-07-25
1,MainScreenAppear,7416695313311560658,1564053102,A,2019-07-25 20:11:42,2019-07-25
2,PaymentScreenSuccessful,3518123091307005509,1564054127,B,2019-07-25 20:28:47,2019-07-25
3,CartScreenAppear,3518123091307005509,1564054127,B,2019-07-25 20:28:47,2019-07-25
4,PaymentScreenSuccessful,6217807653094995999,1564055322,B,2019-07-25 20:48:42,2019-07-25


Проверим количество пользователей и проверим, попадают ли пользователи сразу в несколько групп

Посчитаем количество пользователей в каждой группе

In [13]:
count_users_in_groups = (data
                         .groupby(
                             'group',
                             as_index=False
                         )
                         ['user_id']
                         .nunique()
                         .rename(
                             columns={
                                 'user_id':  'count_users'
                             }
                         )
                         )

count_users_in_groups

,group,count_users
0,A,2489
1,A_1,2520
2,B,2542


Посчитаем долю в процентах

In [14]:
count_users_in_groups['%'] = ((count_users_in_groups['count_users'] /
                              count_users_in_groups['count_users'].sum()*100
                               )
                              .round(2))
count_users_in_groups

,group,count_users,%
0,A,2489,32.96
1,A_1,2520,33.37
2,B,2542,33.66


В идеале группы должны делиться по 33,33% - но посмотрим наскольно отличаются от идеала.

In [15]:
count_users_in_groups['diff_%'] = (count_users_in_groups['%'] -
                                   (1 / len(count_users_in_groups) * 100)
                                   ).round(2)


count_users_in_groups

,group,count_users,%,diff_%
0,A,2489,32.96,-0.37
1,A_1,2520,33.37,0.04
2,B,2542,33.66,0.33


группы незначительно различаются - максимально:  менее 0.4%

Проверим на пересечие групп

In [16]:
count_users_a_b = len(
    set(
        data
        .query('group=="A"')['user_id']
    )
    &
    set(
        data
        .query('group=="B"')['user_id']
    )
)


count_users_a1_b = len(
    set(
        data.query('group=="A_1"')['user_id']
    )
    &
    set(
        data.query('group=="B"')['user_id'])
)



count_users_a_a1 = len(
    set(
        data.query('group=="A"')['user_id']
    )
    &
    set(
        data.query('group=="A_1"')['user_id']
    )
)

In [17]:
print('число пользователей в обеих группах')

for n, g in zip(
    (
        'A и A_1',
        'A и B',
        'A_1 и B'),
    (
        count_users_a_a1,
        count_users_a_b,
        count_users_a1_b
    )
):

    print(f"{n}: {g}".rjust(11))

число пользователей в обеих группах
 A и A_1: 0
   A и B: 0
 A_1 и B: 0


группы точно не пересекаются между собой

Посчитаем число событий и пользователей в логе

In [18]:
print(f'всего событий в логе: {len(data)}')
print(f"всего пользователей в логе: {data['user_id'].nunique()}")

всего событий в логе: 243713
всего пользователей в логе: 7551


найдем среднее кол-во событий на пользователя

In [19]:
count_event_on_users = (data
                        .groupby('user_id')
                        .agg({'event_name': 'count'})
                        .rename(columns={'event_name': 'count_events'})
                        .reset_index()
                        )


count_event_on_users.head()

,user_id,count_events
0,6888746892508752,1
1,6909561520679493,5
2,6922444491712477,47
3,7435777799948366,6
4,7702139951469979,137


In [20]:
count_event_on_users.count_events.describe().reset_index()

,index,count_events
0,count,7551.000000
1,mean,32.275593
2,std,65.154219
3,min,1.000000
4,25%,9.000000
5,50%,20.000000
6,75%,37.000000
7,max,2307.000000


в среднем на пользователя приходится 32 события, по медиане 20,  а максимально - 2307.

Узнаем период за который взяты данные.

In [21]:
print(f'период: {data.date.dt.date.min()} - {data.date.dt.date.max()}')

период: 2019-07-25 - 2019-08-08


А теперь посчитаем число событий по дате в разрезе групп

In [22]:
group_date_count_event = (data
                          .groupby(
                              ['group', 'date'],
                              as_index=False
                          )
                          ['event_name']
                          .count()
                          .rename(
                              columns={'event_name': 'count_events'}
                          )
                          )


group_date_count_event.head()

,group,date,count_events
0,A,2019-07-25,3
1,A,2019-07-26,7
2,A,2019-07-27,27
3,A,2019-07-28,25
4,A,2019-07-29,38


In [23]:
px.bar(group_date_count_event,
       x='date',
       y='count_events',
       color='group',
       facet_col='group',
       labels={'group': 'группа',
               'count_events': 'число событий', 'date': 'дата'},
       title='Количество событий от даты по группам эксперимента',
       template='simple_white')

видно что данных за июль почти нет - отсечем его и будем работать с августом.

In [24]:
df_filter = data.loc[data['date'].dt.month > 7]
df_filter.head()

,event_name,user_id,event_timestamp,group,datetime,date
1431,OffersScreenAppear,776222275221644469,1564585215,A_1,2019-08-01 00:00:15,2019-08-01
1432,CartScreenAppear,776222275221644469,1564585218,A_1,2019-08-01 00:00:18,2019-08-01
1433,MainScreenAppear,8325194800804806038,1564585252,A_1,2019-08-01 00:00:52,2019-08-01
1434,OffersScreenAppear,8325194800804806038,1564585257,A_1,2019-08-01 00:00:57,2019-08-01
1435,MainScreenAppear,6610661657143438937,1564585290,A,2019-08-01 00:01:30,2019-08-01


Узнаем сколько данных мы потеряли

In [25]:
total_events = len(data)
total_events_filtered = len(df_filter)

count_users = data['user_id'].nunique()
count_users_filtered = df_filter['user_id'].nunique()

print(f'количество событий после чистки: {total_events_filtered}')

print(
    f'количество уникальных пользователей после чистки: {count_users_filtered}')

print(
    f'\nпотеряно событий после чистки: {total_events - total_events_filtered}',
      '( {:.2f} % )'
      .format(
          (1-total_events_filtered/total_events)*100
      )
      )

print(
    f'потеряно уникальных пользователей после чистки: {count_users - count_users_filtered}',
    '( {:.2f} % )'
    .format(
        (1-count_users_filtered/count_users)*100
    )
)

количество событий после чистки: 242283
количество уникальных пользователей после чистки: 7541

потеряно событий после чистки: 1430 ( 0.59 % )
потеряно уникальных пользователей после чистки: 10 ( 0.13 % )


посчитаем кол-во пользователей в группах

In [26]:
count_users_in_groups_filter = (df_filter
                                .groupby(
                                    'group',
                                    as_index=False)
                                .agg({'user_id': 'nunique'})
                                .rename(
                                    columns={'user_id': 'count_users_filter'}
                                )
                                )


count_users_in_groups_filter

,group,count_users_filter
0,A,2485
1,A_1,2517
2,B,2539


In [27]:
count_users_in_groups.columns

Index(['group', 'count_users', '%', 'diff_%'], dtype='object')

In [28]:
count_u_in_g = count_users_in_groups[['group', 'count_users']]

table_count_users_groups = (count_u_in_g
                            .merge(
                                count_users_in_groups_filter,
                                how='left',
                                on='group')
                            .rename(
                                columns={'count_users': 'count_users_raw'}
                            )
                            )

table_count_users_groups

,group,count_users_raw,count_users_filter
0,A,2489,2485
1,A_1,2520,2517
2,B,2542,2539


Посчитаем насколько и восколько изменилось кол-во пользователей в группах.

In [29]:
table_count_users_groups['diff'] = (table_count_users_groups['count_users_raw'] -
                                    table_count_users_groups['count_users_filter'])



table_count_users_groups['diff_%'] = (
    (
        (1 - table_count_users_groups['count_users_filter'] /
         table_count_users_groups['count_users_raw'])*100
    )
    .round(3)
)

table_count_users_groups

,group,count_users_raw,count_users_filter,diff,diff_%
0,A,2489,2485,4,0.161
1,A_1,2520,2517,3,0.119
2,B,2542,2539,3,0.118


Вывод:   
Присутствуют пользователи всех 3х групп и незначительно отличаются между собой по количеству пользователей.    
После изменения временного  периода данные значительно не изменились.  Данные готовы к дальнейшему анализу.

# Анализ воронок продаж

В таблице события значат следующее

`MainScreenAppear` -появление главного экрана   
`OffersScreenAppear` - появлние экран с предложениями    
`CartScreenAppear` - появление экрана корзины   
`PaymentScreenSuccessful` - появление экрана об успешном платеже   
`Tutorial` - появление экрана с обучением

Для лучшего понимания переведем события с помощью функции

In [30]:
def rename_events(value):
    if value == 'MainScreenAppear':
        return 'главный экран'

    elif value == 'OffersScreenAppear':
        return 'экран с предложениями'

    elif value == 'CartScreenAppear':
        return 'экран корзины'

    elif value == 'PaymentScreenSuccessful':
        return 'экран об оплате платежа'
    else:
        return 'экран с обучением'

запишем перевод в отдельный столбец

In [31]:
df_filter['event_name_rus'] = df_filter['event_name'].apply(rename_events)
df_filter.head()

,event_name,user_id,event_timestamp,group,datetime,date,event_name_rus
1431,OffersScreenAppear,776222275221644469,1564585215,A_1,2019-08-01 00:00:15,2019-08-01,экран с предложениями
1432,CartScreenAppear,776222275221644469,1564585218,A_1,2019-08-01 00:00:18,2019-08-01,экран корзины
1433,MainScreenAppear,8325194800804806038,1564585252,A_1,2019-08-01 00:00:52,2019-08-01,главный экран
1434,OffersScreenAppear,8325194800804806038,1564585257,A_1,2019-08-01 00:00:57,2019-08-01,экран с предложениями
1435,MainScreenAppear,6610661657143438937,1564585290,A,2019-08-01 00:01:30,2019-08-01,главный экран


посмотрим , сколько раз пользователи совершали события

In [32]:
ev_raw = (df_filter
          .groupby('event_name_rus', as_index=False)
          .agg(
              {'user_id': 'count'}
          )
          .sort_values(
              'user_id', ascending=False
          )
          .rename(columns={
              'user_id': 'count_users'
          }
          )
          )


ev_raw

,event_name_rus,count_users
0,главный экран,118171
4,экран с предложениями,46628
1,экран корзины,42461
2,экран об оплате платежа,34013
3,экран с обучением,1010


Рассчитаем долю пользователей совершивщих одно событие. Для начала найдем сколько пользователи совершали событий

In [33]:
count_event_on_users_filter = (df_filter
                               .groupby('user_id')
                               .agg(
                                   {'event_name': 'count'}
                               )
                               .rename(
                                   columns={
                                       'event_name': 'count_events'
                                   }
                               )
                               .reset_index()
                               )


count_event_on_users.head()

,user_id,count_events
0,6888746892508752,1
1,6909561520679493,5
2,6922444491712477,47
3,7435777799948366,6
4,7702139951469979,137


Затем найдем пользователей совершивших 1 событие

In [34]:
users_with_1_event = (count_event_on_users_filter
                      .query('count_events==1')['user_id']
                      )

In [35]:
print('доля пользователей совершивших одно событие:',
      '{:.2f} %'
      .format(
          users_with_1_event.count() /
          len(df_filter)*100
      )
      )

доля пользователей совершивших одно событие: 0.05 %


Вывод: пользователей совершивших одно событие ничтожно мало

Теперь проверим , а действительно ли событие ГЛАВНЫЙ ЭКРАН вляется первым и вообще в какие последовательности выстраиваются события

Найдем события совершенные пользователями за каждый день

In [36]:
ev = (df_filter
      .groupby(['user_id', 'date'])
      ['event_name_rus']
      .unique()
      .reset_index()
      )


ev.head()

,user_id,date,event_name_rus
0,6888746892508752,2019-08-06,[главный экран]
1,6909561520679493,2019-08-07,"[главный экран, экран об оплате платежа, экран корзины, экран с предложениями]"
2,6922444491712477,2019-08-04,"[главный экран, экран об оплате платежа, экран корзины, экран с предложениями]"
3,6922444491712477,2019-08-05,"[главный экран, экран об оплате платежа, экран корзины, экран с предложениями]"
4,6922444491712477,2019-08-06,"[главный экран, экран с предложениями, экран об оплате платежа, экран корзины]"


Выташим первое событие

In [37]:
ev['screen_1'] = ev['event_name_rus'].apply(lambda x: x[0])
ev.head()

,user_id,date,event_name_rus,screen_1
0,6888746892508752,2019-08-06,[главный экран],главный экран
1,6909561520679493,2019-08-07,"[главный экран, экран об оплате платежа, экран корзины, экран с предложениями]",главный экран
2,6922444491712477,2019-08-04,"[главный экран, экран об оплате платежа, экран корзины, экран с предложениями]",главный экран
3,6922444491712477,2019-08-05,"[главный экран, экран об оплате платежа, экран корзины, экран с предложениями]",главный экран
4,6922444491712477,2019-08-06,"[главный экран, экран с предложениями, экран об оплате платежа, экран корзины]",главный экран


заодно найдем из скольких шагов состоит последовательность 

In [38]:
ev['count_steps'] = ev['event_name_rus'].apply(lambda x: len(x))
ev.head()

,user_id,date,event_name_rus,screen_1,count_steps
0,6888746892508752,2019-08-06,[главный экран],главный экран,1
1,6909561520679493,2019-08-07,"[главный экран, экран об оплате платежа, экран корзины, экран с предложениями]",главный экран,4
2,6922444491712477,2019-08-04,"[главный экран, экран об оплате платежа, экран корзины, экран с предложениями]",главный экран,4
3,6922444491712477,2019-08-05,"[главный экран, экран об оплате платежа, экран корзины, экран с предложениями]",главный экран,4
4,6922444491712477,2019-08-06,"[главный экран, экран с предложениями, экран об оплате платежа, экран корзины]",главный экран,4


In [39]:
ev['count_steps'].value_counts()

1    11335
4     8689
2     4565
3     1550
5      363
Name: count_steps, dtype: int64

В основном совершается 1 событие и 4

In [40]:

ev.query('count_steps>1').screen_1.value_counts().reset_index()

,index,screen_1
0,главный экран,12264
1,экран с предложениями,1139
2,экран с обучением,807
3,экран об оплате платежа,588
4,экран корзины,369


А первым экраном что они видят является главный экран приложения (он же главный)

и посчитаем количество уникальных выполненыых сценариев .И посмотрим на 10 из них.    
Для быстрых вычислений превратим списки в строки

In [41]:
ev['steps'] = ev['event_name_rus'].apply(lambda x: '__'.join(x))

In [42]:
table_steps = (ev
               .query('count_steps>=4')
               ['steps']
               .value_counts()
               .reset_index()
               )


table_steps.head()

,index,steps
0,главный экран__экран об оплате платежа__экран корзины__экран с предложениями,2607
1,главный экран__экран корзины__экран об оплате платежа__экран с предложениями,1797
2,главный экран__экран с предложениями__экран об оплате платежа__экран корзины,1645
3,главный экран__экран с предложениями__экран корзины__экран об оплате платежа,1373
4,главный экран__экран корзины__экран с предложениями__экран об оплате платежа,298


In [43]:


table_steps['%'] = (
    (
        table_steps['steps'] /
        table_steps['steps'].sum()*100
    )
    .round(2)
)

table_steps.head(10)

,index,steps,%
0,главный экран__экран об оплате платежа__экран корзины__экран с предложениями,2607,28.80
1,главный экран__экран корзины__экран об оплате платежа__экран с предложениями,1797,19.85
2,главный экран__экран с предложениями__экран об оплате платежа__экран корзины,1645,18.17
3,главный экран__экран с предложениями__экран корзины__экран об оплате платежа,1373,15.17
4,главный экран__экран корзины__экран с предложениями__экран об оплате платежа,298,3.29
5,экран с обучением__главный экран__экран с предложениями__экран корзины__экран об оплате платежа,182,2.01
6,экран с предложениями__главный экран__экран об оплате платежа__экран корзины,161,1.78
7,экран с предложениями__экран об оплате платежа__экран корзины__главный экран,112,1.24
8,экран об оплате платежа__экран корзины__экран с предложениями__главный экран,108,1.19
9,экран об оплате платежа__экран корзины__главный экран__экран с предложениями,104,1.15


Все-таки пользователи совершают события в разной последовательности , но начинают с главного экрана.   
А 30% с главного экрана переходят а экран оплаты затем в корзину и потом смотрят предложения.


С помощью графика посмотрим сколько пользоватлеей находится на каждом событии. Только исключим `экран с обучением` т.к. он появляется где угодно, а основной последовательностью для пользователя будем считать    
`главный экран -> экран с предложениями -> экран корзины  -> экран об оплате платежа`

In [44]:
event_users = (df_filter
               .groupby(
                   'event_name_rus',
                   as_index=False
               )
               ['user_id']
               .nunique()
               .rename(
                   columns={
                       'user_id': 'count_users'
                   }
               )
               )



event_users.reset_index()

,index,event_name_rus,count_users
0,0,главный экран,7427
1,1,экран корзины,3738
2,2,экран об оплате платежа,3541
3,3,экран с обучением,843
4,4,экран с предложениями,4602


Выставляем порядок событий

In [45]:
event_users.index = [0, 2, 3, 4, 1]
event_users = event_users.sort_index()
event_users

,event_name_rus,count_users
0,главный экран,7427
1,экран с предложениями,4602
2,экран корзины,3738
3,экран об оплате платежа,3541
4,экран с обучением,843


In [46]:
ev_raw_without_tutorial = (event_users
                           .query(
                               "event_name_rus!='экран с обучением'"
                           )
                           )

Строим график

In [47]:
px.funnel(ev_raw_without_tutorial,
          y='event_name_rus',
          x='count_users',
          labels={'event_name_rus': 'события',
                  'count_users':  'число пользователей'},
            title='Количество пользователей на разных событиях',
            template='simple_white')

видно что на главном экране задерживаютмсся пользователями и дальше не переходят. Узнаем сколько это в процентах

Сдвигаем число пользвателей на 1 и находим отношение

In [48]:
ev_raw_without_tutorial['shift'] = (ev_raw_without_tutorial['count_users']
                                    .shift(1)
                                    )



ev_raw_without_tutorial

,event_name_rus,count_users,shift
0,главный экран,7427,NaN
1,экран с предложениями,4602,7427.0
2,экран корзины,3738,4602.0
3,экран об оплате платежа,3541,3738.0


In [49]:
ev_raw_without_tutorial['%'] = (
    (
        ev_raw_without_tutorial['count_users'] /
        ev_raw_without_tutorial['shift']*100
    )
    .round(2)
)



ev_raw_without_tutorial

,event_name_rus,count_users,shift,%
0,главный экран,7427,NaN,NaN
1,экран с предложениями,4602,7427.0,61.96
2,экран корзины,3738,4602.0,81.23
3,экран об оплате платежа,3541,3738.0,94.73


Вывод: с главного экрана на экран с  предоложениями переходит всего 60% пользователей. 

Найдем долю пользователей прошедших от первого события до оплаты.У них должно быть событие `экран об оплате платежа`

In [50]:
event = 'экран об оплате платежа'

найдем это событиеи сгруппируем по уникальным пользоваиелям

In [51]:
ev['step_pay'] = ev['event_name_rus'].apply(lambda x: event in x)


ev_pay = ev.groupby('user_id')['step_pay'].unique().reset_index()

ev_pay.head()

,user_id,step_pay
0,6888746892508752,[False]
1,6909561520679493,[True]
2,6922444491712477,"[True, False]"
3,7435777799948366,[False]
4,7702139951469979,"[False, True]"


In [52]:
ev_pay['step_pay'] = ev_pay['step_pay'].apply(sum)
ev_pay.head()

,user_id,step_pay
0,6888746892508752,0
1,6909561520679493,1
2,6922444491712477,1
3,7435777799948366,0
4,7702139951469979,1


Тогда

In [53]:
print('доля оплативших пользователей:',
      '{:.2f} %'
      .format(
          ev_pay.step_pay.sum() / len(ev_pay)*100
      )
      )

доля оплативших пользователей: 46.96 %


Вывод: примерно 47 % пользователей заказывают продукты.

Рекомендации:   
1. провести исследования (интервью, опросы, анализ метрик) для понимания причин, почему 40% пользователей не переходят дальше.

2. Оптимизация интерфейса: Убедитесь, что элементы интерфейса понятны и бросаются в глаза а информация на экраня ясная и убедительная .    
Проверьте, достаточно ли привлекательны кнопки или ссылки для перехода.

4. Проверить, что переход на следующий шаг легкий и интуитивно понятный.

# Анализ результатов A/A/B-теста

Проверим есть ли разница между выборками контрольных групп А и А_1

посчитаем количество число событий совершенное каждыми пользователи

In [54]:
count_events_all_groups = (df_filter
                           .groupby(
                               ['group', 'user_id'],
                               as_index=False
                           )
                           ['event_name']
                           .count()
                           .rename(
                               columns={
                                   'event_name': 'count_events'
                               }
                           )
                           )


count_events_all_groups.head()

,group,user_id,count_events
0,A,6888746892508752,1
1,A,6922444491712477,47
2,A,8740973466195562,9
3,A,12692216027168046,10
4,A,15708180189885246,126


Посчитаем число пользователей , совершивших события в каждой из контрольных групп и долю пользователей совершивших событие

Сформулируем гипотезы

In [55]:
hipotesys_1 = {
    'H_0': 'среднее количество пользователей  обеих групп совершивших событие одинаковое',
    'H_1': 'среднее количество пользователей обеих групп совершивших событие различается '
}

alpha = .05

на подмогу пишем фунцию, которая выведет "карточки" с событием , долями, и результатми Z-теста . Применим поправку Бонсферонни т.к. проводится будет несеолько тестов.

Начнем с поправки Бонсферони

планируется провестти тесты :  
 - между контрольными группами - 1    
 - между каждой контрольной группой и экспериментальной - 2   
 - между обьединенными контрольными и экспериментальной - 1   
Итого:  4 теста.

In [56]:
count_tests=4

тогда поправка составит

In [57]:
alpha_corr = alpha / count_tests
print("Откорректированное alpha =", alpha_corr)

Откорректированное alpha = 0.0125


подготовим данные для функции. Соберем таблицу с пользователями по событию в группах

In [58]:
groups_events_count_users = (df_filter
                             .groupby(
                                 ['group', 'event_name_rus']
                             )['user_id']
                             .nunique()
                             .reset_index()
                             .rename(
                                 columns={
                                     'user_id': 'count_users'
                                 }
                             )
                             )


groups_events_count_users

,group,event_name_rus,count_users
0,A,главный экран,2451
1,A,экран корзины,1266
2,A,экран об оплате платежа,1200
3,A,экран с обучением,278
4,A,экран с предложениями,1543
5,A_1,главный экран,2479
6,A_1,экран корзины,1239
7,A_1,экран об оплате платежа,1158
8,A_1,экран с обучением,284
9,A_1,экран с предложениями,1526


обьединим со 2й таблицей (с количеством пользователй в каждой группе)

In [59]:
groups_events_count_users = (groups_events_count_users.
                             merge(
                                 table_count_users_groups[
                                     [
                                         'group',
                                         'count_users_filter'
                                     ]
                                 ],
                                 how='left',
                                 on='group'
                             )
                             )


groups_events_count_users.rename(
    columns={
        'count_users': 'count_users_event',
        'count_users_filter': 'count_users_group'
    },
    inplace=True
)


groups_events_count_users

,group,event_name_rus,count_users_event,count_users_group
0,A,главный экран,2451,2485
1,A,экран корзины,1266,2485
2,A,экран об оплате платежа,1200,2485
3,A,экран с обучением,278,2485
4,A,экран с предложениями,1543,2485
5,A_1,главный экран,2479,2517
6,A_1,экран корзины,1239,2517
7,A_1,экран об оплате платежа,1158,2517
8,A_1,экран с обучением,284,2517
9,A_1,экран с предложениями,1526,2517


посчитаем долю пользователей по событию в группах

In [60]:
groups_events_count_users['user_share'] = (
    groups_events_count_users['count_users_event'] /
    groups_events_count_users['count_users_group']
)

groups_events_count_users

,group,event_name_rus,count_users_event,count_users_group,user_share
0,A,главный экран,2451,2485,0.986318
1,A,экран корзины,1266,2485,0.509457
2,A,экран об оплате платежа,1200,2485,0.482897
3,A,экран с обучением,278,2485,0.111871
4,A,экран с предложениями,1543,2485,0.620926
5,A_1,главный экран,2479,2517,0.984903
6,A_1,экран корзины,1239,2517,0.492253
7,A_1,экран об оплате платежа,1158,2517,0.460072
8,A_1,экран с обучением,284,2517,0.112833
9,A_1,экран с предложениями,1526,2517,0.606277


In [61]:
def results_Z_test(list_events, list_groups,  hipotesys, alpha=alpha_corr):
    '''    
    Выводит:    
    1. событие   
    Результаты Z-теста
    2. p-value      
    3. доли пользователей групп по событию
    4. принимаемую гипотезу'''

    # делаем группы ----------
    _ = []
    for i in list_groups:
        _.append(groups_events_count_users.query("group==@i"))

    group_1, group_2 = tuple(_)
    # -----------------------------------

    # расчет Z-теста ------------------------

    for ev in list_events:

        count_users_event_1 = int(
            group_1.query(
                'event_name_rus==@ev')['count_users_event'])

        count_users_event_2 = int(
            group_2.query(
                'event_name_rus==@ev')['count_users_event'])

        total_1 = int(
            group_1.query(
                'event_name_rus==@ev')['count_users_group'])

        total_2 = int(
            group_2.query(
                'event_name_rus==@ev')['count_users_group'])

        p1 = count_users_event_1/total_1
        p2 = count_users_event_2/total_2

        p_combined = (count_users_event_1 +
                      count_users_event_2) / (total_1 + total_2)

        difference = p1-p2

        z_value = (difference /
                   m.sqrt(
                       p_combined * (1-p_combined) * (1/total_1 + 1/total_2)
                   )
                   )

        distr = stat.norm(0, 1)

        p_value = (1 - distr.cdf(abs(z_value)))*2

        # карточка
        # ------------------------------------------------
        print('-//'*30, '\n\nсобытие: ',  ev.upper(), '\n')

        print('доля пользователей совершивших событие')
        for g, p in zip(list_groups, (p1, p2)):
            print(
                f'в группе {g}:', '{:.2f} %'.format(p*100))

        print("\np_value = {:.3f}".format(p_value))

        if p_value > alpha:
            print("принимаем гипотезу:", hipotesys.get('H_0'))
        else:
            print("принимаем гипотезу:", hipotesys.get('H_1'))
        # -------------------------------------------------

список событий по которым будем считать

In [62]:
list_events = list(event_users['event_name_rus'])
list_events

['главный экран',
 'экран с предложениями',
 'экран корзины',
 'экран об оплате платежа',
 'экран с обучением']

## Анализ между контрольными группами

Данный анализ поможет измерить качество деления, определить доли групп, и подтвердить (или опровергнуть )однородность трафика — отсутствие различия ключевых метрик между группами, которые видят абсолютно одинаковые версии продукта.

Расчитаем


In [63]:
results_Z_test(list_events=list_events,
               list_groups=['A', 'A_1'],
              
               hipotesys=hipotesys_1)

-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-// 

событие:  ГЛАВНЫЙ ЭКРАН 

доля пользователей совершивших событие
в группе A: 98.63 %
в группе A_1: 98.49 %

p_value = 0.674
принимаем гипотезу: среднее количество пользователей  обеих групп совершивших событие одинаковое
-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-// 

событие:  ЭКРАН С ПРЕДЛОЖЕНИЯМИ 

доля пользователей совершивших событие
в группе A: 62.09 %
в группе A_1: 60.63 %

p_value = 0.287
принимаем гипотезу: среднее количество пользователей  обеих групп совершивших событие одинаковое
-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-// 

событие:  ЭКРАН КОРЗИНЫ 

доля пользователей совершивших событие
в группе A: 50.95 %
в группе A_1: 49.23 %

p_value = 0.224
принимаем гипотезу: среднее количество пользователей  обеих групп совершивших событие одинаковое
-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-/

Вывод: Контрольные группы незначиельно отличаются  между собой по всем событиям - это подтверждает что трафик однородный в рамках контрольных групп, а значит можно сделать анализ между контрольными группами и экспериментальной.


## Анализ между первой контрольной и экспериментальной группами

In [64]:
results_Z_test(list_events=list_events,
               list_groups=['A', 'B'],
               hipotesys=hipotesys_1)

-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-// 

событие:  ГЛАВНЫЙ ЭКРАН 

доля пользователей совершивших событие
в группе A: 98.63 %
в группе B: 98.35 %

p_value = 0.406
принимаем гипотезу: среднее количество пользователей  обеих групп совершивших событие одинаковое
-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-// 

событие:  ЭКРАН С ПРЕДЛОЖЕНИЯМИ 

доля пользователей совершивших событие
в группе A: 62.09 %
в группе B: 60.38 %

p_value = 0.212
принимаем гипотезу: среднее количество пользователей  обеих групп совершивших событие одинаковое
-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-// 

событие:  ЭКРАН КОРЗИНЫ 

доля пользователей совершивших событие
в группе A: 50.95 %
в группе B: 48.56 %

p_value = 0.091
принимаем гипотезу: среднее количество пользователей  обеих групп совершивших событие одинаковое
-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-/

Вывод: экспериментальная группа В по событиям почти не отличается от контрольной группы А


## Анализ между 2ой контрольной и экспериментальной группами

In [65]:
results_Z_test(list_events=list_events,
               list_groups=['A_1', 'B'],
               hipotesys=hipotesys_1)

-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-// 

событие:  ГЛАВНЫЙ ЭКРАН 

доля пользователей совершивших событие
в группе A_1: 98.49 %
в группе B: 98.35 %

p_value = 0.681
принимаем гипотезу: среднее количество пользователей  обеих групп совершивших событие одинаковое
-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-// 

событие:  ЭКРАН С ПРЕДЛОЖЕНИЯМИ 

доля пользователей совершивших событие
в группе A_1: 60.63 %
в группе B: 60.38 %

p_value = 0.856
принимаем гипотезу: среднее количество пользователей  обеих групп совершивших событие одинаковое
-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-// 

событие:  ЭКРАН КОРЗИНЫ 

доля пользователей совершивших событие
в группе A_1: 49.23 %
в группе B: 48.56 %

p_value = 0.637
принимаем гипотезу: среднее количество пользователей  обеих групп совершивших событие одинаковое
-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-/

Вывод: экспериментальная группа В не отличается от контрольной группы А_1 .

## Анализ между обьединенными контрольными группами и экспериментальной

а теперь сравним группу В с обьединенными контрольными группами.

назовем одним именем контрольные группы

In [66]:
df_filter['group'].replace(['A', 'A_1'], 'big_A', inplace=True)
df_filter['group'].unique()

array(['big_A', 'B'], dtype=object)

состаим таблицу по которой посчитаем Z-тест 

Посчитаем число пользователей

In [67]:
table_count_users_groups_2 = (df_filter
                              .groupby('group')
                              ['user_id']
                              .nunique()
                              .reset_index()
                              .rename(
                                  columns=dict(
                                      user_id='count_users_group'
                                  )
                              )
                              )


table_count_users_groups_2

,group,count_users_group
0,B,2539
1,big_A,5002


посчитаем число пользователей по событиям групп

In [68]:
groups_events_count_users = (df_filter
                             .groupby(
                                 ['group', 'event_name_rus']
                             )
                             ['user_id']
                             .nunique()
                             .reset_index()
                             .rename(columns={
                                 'user_id':  'count_users'
                             })
                             )


groups_events_count_users

,group,event_name_rus,count_users
0,B,главный экран,2497
1,B,экран корзины,1233
2,B,экран об оплате платежа,1183
3,B,экран с обучением,281
4,B,экран с предложениями,1533
5,big_A,главный экран,4930
6,big_A,экран корзины,2505
7,big_A,экран об оплате платежа,2358
8,big_A,экран с обучением,562
9,big_A,экран с предложениями,3069


обьединим эти таблицы в одну 

In [69]:
groups_events_count_users = (groups_events_count_users
                             .merge(
                                 table_count_users_groups_2,
                                 how='left',
                                 on='group'
                             )
                             )

groups_events_count_users.rename(
    columns=dict(count_users='count_users_event'),
    inplace=True
)

groups_events_count_users

,group,event_name_rus,count_users_event,count_users_group
0,B,главный экран,2497,2539
1,B,экран корзины,1233,2539
2,B,экран об оплате платежа,1183,2539
3,B,экран с обучением,281,2539
4,B,экран с предложениями,1533,2539
5,big_A,главный экран,4930,5002
6,big_A,экран корзины,2505,5002
7,big_A,экран об оплате платежа,2358,5002
8,big_A,экран с обучением,562,5002
9,big_A,экран с предложениями,3069,5002


и найдем долю пользователей в событиии

In [70]:
groups_events_count_users['user_share'] = (groups_events_count_users['count_users_event'] /
                                           groups_events_count_users['count_users_group'])

groups_events_count_users

,group,event_name_rus,count_users_event,count_users_group,user_share
0,B,главный экран,2497,2539,0.983458
1,B,экран корзины,1233,2539,0.485624
2,B,экран об оплате платежа,1183,2539,0.465931
3,B,экран с обучением,281,2539,0.110673
4,B,экран с предложениями,1533,2539,0.603781
5,big_A,главный экран,4930,5002,0.985606
6,big_A,экран корзины,2505,5002,0.500800
7,big_A,экран об оплате платежа,2358,5002,0.471411
8,big_A,экран с обучением,562,5002,0.112355
9,big_A,экран с предложениями,3069,5002,0.613555


применим функцию к этой таблице

In [71]:
results_Z_test(list_events=list_events,
               list_groups=['big_A','B'],
               hipotesys=hipotesys_1)

-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-// 

событие:  ГЛАВНЫЙ ЭКРАН 

доля пользователей совершивших событие
в группе big_A: 98.56 %
в группе B: 98.35 %

p_value = 0.470
принимаем гипотезу: среднее количество пользователей  обеих групп совершивших событие одинаковое
-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-// 

событие:  ЭКРАН С ПРЕДЛОЖЕНИЯМИ 

доля пользователей совершивших событие
в группе big_A: 61.36 %
в группе B: 60.38 %

p_value = 0.411
принимаем гипотезу: среднее количество пользователей  обеих групп совершивших событие одинаковое
-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-// 

событие:  ЭКРАН КОРЗИНЫ 

доля пользователей совершивших событие
в группе big_A: 50.08 %
в группе B: 48.56 %

p_value = 0.213
принимаем гипотезу: среднее количество пользователей  обеих групп совершивших событие одинаковое
-//-//-//-//-//-//-//-//-//-//-//-//-//-//-/

Вывод: изменение шрифта значительно не повлияло на пользователей приложения.

# Итоговый вывод

Сделано:   
отформатировали названия столбцов,    
 удалили дубликаты,    
  привели данные к нужному типу    
   и создали столбцы с  датой и временем      
   и просто датой.     
   Переименовали группы,      
   рассчитали период за который были взяты данные,     
    нашли аномалии и отфильтровали их,     
    рассчитали долю потерянных событий и пользователей     
    и рассчитали число порльзователей в группах.   

Перевели события, посмотрели в каких последовательнотях они могут идти, нашли событие на которм отсеиваются больше всего пользователей, построили вороноку событий и нашли конверсию пользователей на каждом событиии. 
Узнали размеры групп и проверили их на пересечение.
Затем исследовали результаты ААВ теста с помощью теста Манна-Уитни. 

Выяснили:    
 - данные взяты с 25-07-2019 - по 08-08-2019   
 - что 95% пользователей совершают менее 89 событий
 - данных за июль почти нет- поэто му взяты данные за август
 - группы после очистки от аномалий группы незначительно отличаются по количеству пользователей
 
 - что последовательности событий начинаются в основном с главнеого экрана , но далее могут идти в разном порядке   
 - что с главного экрана переходит всего 60% пользователей   
 - контрольные группы и экспериментальная  между собой не пересекаются  

 Рекомендации по поводу главного экрана:   
- провести исследования (интервью, опросы, анализ метрик) для понимания причин, почему 40% пользователей не переходят дальше.
-  Оптимизация интерфейса: Убедитесь, что элементы интерфейса понятны и бросаются в глаза а информация на экраня ясная и убедительная .    
Проверьте, достаточно ли привлекательны кнопки или ссылки для перехода.
- Проверить, что переход на следующий шаг легкий и интуитивно понятный.
   

 Применив тест Z-тест и поправку Бонсферони, выяснили: 
 - разниц между контрольными группами по каждому событию нет
 - разниц между группой с измененным шрифтом  и каждой контрольной котрольной группой по каждому событию нет
 - разниц между группой с измененным шрифтом и обьединенными контрольными группами нет



Итог:
- примерно 60% пользователей переходят с главного экрана на экран с предложениями, 
- с экрана с предложениями переходит на экран корзины примерно 80% 
- а с корзины до оплаты доходит 94 %.
- пользователи совершают события в разной последовательности , но начинают с главного экрана.   
- А 30% с главного экрана переходят а экран оплаты затем в корзину и потом смотрят предложения.
- Изменение шрифта в приложении не повлияло на пользователей.